<a href="https://colab.research.google.com/github/unknown-jun/First_step_of_data_analist/blob/master/Titanic_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 필요한 라이브러리들을 로드
import pandas as pd
import numpy as np
import re as re
from sklearn.linear_model import LogisticRegression

In [2]:
# 파일 직접 업로드하기
from google.colab import files
myfile = files.upload()

Saving sample_submission.csv to sample_submission (5).csv
Saving test.csv to test (5).csv
Saving train.csv to train (5).csv


In [3]:
# 파일 불러오기
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

In [4]:
# 데이터 살펴보기
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [6]:
# SibSp : 동반한 형제, 자매, 배우자 수
# Parch : 동반한 부모, 자식 수
# Cabin : 객실 번호
# Embarked : 승선한 항구명

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


# **데이터 정제하기**

In [8]:
# SibSp와 Parch로 새로운 변수 FamilySize 생성하기
train['FamilySize'] = train['SibSp'] + train['Parch'] + 1
test['FamilySize'] = test['SibSp'] + test['Parch'] + 1

In [9]:
# 혼자 탑승했는지 여부를 알려줄 수 있는 변수 alone 생성하기
train['IsAlone'] = 0
train.loc[train['FamilySize'] == 1, 'IsAlone'] = 1

test['IsAlone'] = 0
test.loc[train['FamilySize'] == 1, 'IsAlone'] = 1

In [10]:
# Embarked의 결측치를 빈도가 가장 많은 'S'로 대체하기
train['Embarked'] == train['Embarked'].fillna('S')
test['Embarked'] == test['Embarked'].fillna('S')

0      True
1      True
2      True
3      True
4      True
       ... 
413    True
414    True
415    True
416    True
417    True
Name: Embarked, Length: 418, dtype: bool

In [11]:
# Fare의 결측치를 중앙값으로 대체하기
train['Fare'] = train['Fare'].fillna(train['Fare'].median())
test['Fare'] = test['Fare'].fillna(test['Fare'].median())

In [12]:
# Age는 정규분포를 통하여 결측값을 처리하기
age_avg = train['Age'].mean()
age_std = train['Age'].std()
age_null_count = train['Age'].isnull().sum()
age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size = age_null_count)
train['Age'][np.isnan(train['Age'])] = age_null_random_list
train['Age'] = train['Age'].astype(int)

age_avg = test['Age'].mean()
age_std = test['Age'].std()
age_null_count = test['Age'].isnull().sum()
age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size = age_null_count)
test['Age'][np.isnan(test['Age'])] = age_null_random_list
test['Age'] = test['Age'].astype(int)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [13]:
# Name에서 호칭 추출하기
train['Name'] = train['Name'].str.extract('([A-Za-z] +)\.', expand=False)
test['Name'] = test['Name'].str.extract('([A-Za-z] +)\.', expand=False)

In [14]:
# 적은 수의 호칭은 'Rare'로 대체
train['Name'] = train['Name'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
train['Name'] = train['Name'].replace('Mlle', 'Miss')
train['Name'] = train['Name'].replace('Ms', 'Miss')
train['Name'] = train['Name'].replace('Mme', 'Mrs')

test['Name'] = test['Name'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
test['Name'] = test['Name'].replace('Mlle', 'Miss')
test['Name'] = test['Name'].replace('Ms', 'Miss')
test['Name'] = test['Name'].replace('Mme', 'Mrs')

In [15]:
# 성별을 숫자형으로 변환
train['Sex'] = train['Sex'].map( {'female': 0, 'male': 1} ).astype(int)

# 호칭을 숫자형으로 변환
name_mapping = {"Mr":1, "Miss":2, "Mrs":3, "Master":4, "Rare":5}
train['Name'] = train['Name'].map(name_mapping)
train['Name'] = train['Name'].fillna(0)


In [16]:
# Embarked에 있는 2개의 결측치를 최빈값으로 대체
train['Embarked'].value_counts(dropna=True).idxmax()
most_freq = train['Embarked'].value_counts(dropna=True).idxmax()
train['Embarked'].fillna(most_freq, inplace = True)

In [17]:
# Embarked를 숫자형으로 변환
train['Embarked'] = train['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

# Fare를 범주형으로 변환
train.loc[train['Fare'] <= 7.91, 'Fare'] = 0
train.loc[(train['Fare'] >= 14.454), 'Fare'] = 1
train.loc[(train['Fare'] >= 31), 'Fare'] = 2
train.loc[(train['Fare'] > 31), 'Fare'] = 3
train['Fare'] = train['Fare'].astype(int)

# Age를 범주형으로 변환
train.loc[train['Age'] <= 16, 'Age'] = 0
train.loc[train['Age'] >= 32, 'Age'] = 1
train.loc[train['Age'] >= 48, 'Age'] = 2
train.loc[train['Age'] >= 64, 'Age'] = 3
train.loc[train['Age'] > 64, 'Age'] = 4

In [17]:
# Sex를 숫자형으로 변환
test['Sex'] = test['Sex'].map( {'female':0, 'male':1} ).astype(int)

# Name을 숫자형으로 변환
name_mapping = {"Mr": 1, "Miss":2, "Mrs":3, "Master":4, "Rare":5}
test['Name'] = test['Name'].mape( {name_mapping})
test['Name'] = test['Name'].fillna(0)

# 